In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
import dateutil.parser
from datetime import date
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

import seaborn as sns
import matplotlib.pyplot as plt

import pickle
import time

plt.style.use("seaborn")
%matplotlib inline

%load_ext autotime

In [4]:
link='https://www.sephora.com/product/pro-filtr-soft-matte-longwear-foundation-P87985432?skuId=1925486&icid2=products%20grid:p87985432'

time: 684 µs


In [5]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

time: 1.22 ms


In [6]:
driver = webdriver.Chrome(chromedriver)
driver.get(link)
time.sleep(1);

time: 11.1 s


In [7]:
no_account_button = driver.find_element_by_class_name('css-1xibwal')
no_account_button.click()

time: 3.82 s


In [8]:
driver.execute_script("window.scrollTo(0, 1000)")
time.sleep(10);

time: 10 s


In [9]:
temp_cols = ['star', 'other']
temp_df=pd.DataFrame(columns=temp_cols, index=[])

time: 56.8 ms


In [10]:
i=1
j=7
for reviews in range(1750):
    for number in range(i,j):
        row={}
        path= f".//*[@id='ratings-reviews']/div[2]/div[3]/div[{number}]"
        load_reviews=driver.find_element_by_xpath(path)
        row['other']=load_reviews.text
        star=load_reviews.find_element_by_class_name("css-17ol29l").get_attribute('style')
        row['star']=star
        temp_df=temp_df.append(row, ignore_index=True)
    i=j
    j=j+5
    next_six = driver.find_element_by_class_name('css-end8gr')
    next_six.click()
    time.sleep(3);
    with open('temp.pkl', 'wb') as picklefile:
        pickle.dump(temp_df, picklefile)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//*[@id='ratings-reviews']/div[2]/div[3]/div[2047]"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.4 x86_64)


time: 6h 58min 29s


In [13]:
temp_df.head()

,star,other
0,width: 100%;,shaaliyah\nEye Color Brown\nHair color Black\n...
1,width: 100%;,bloodyblondie\nAge 18-24\nEye Color Blue\nHair...
2,width: 100%;,Shortcake92\nAge 18-24\nEye Color Brown\nHair ...
3,width: 100%;,JDhome89\nEye Color Brown\nHair color Brunette...
4,width: 100%;,creecreex\nEye Color Brown\nHair color Brunett...


time: 43.9 ms


In [14]:
temp_df.star.unique()

array(['width: 100%;', 'width: 80%;', 'width: 60%;', 'width: 20%;',
       'width: 40%;'], dtype=object)

time: 85.6 ms


In [15]:
temp_df.shape

(2046, 2)

time: 4.3 ms


In [16]:
cols = ['username','eye_color', 'hair', 'skin_tone', 'skin_type',
                 'skin_concerns', 'foundation_color', 'review', 'recommend',
                'unhelpful', 'helpful']
df=pd.DataFrame(columns=cols, index=[])

time: 37.9 ms


In [17]:
for info in temp_df.other:
    seperations={}
    seperations={'username':'Unknown','eye_color':'Unknown', 'hair':'Unknown', 'skin_tone':'Unknown', 'skin_type':'Unknown',
                 'skin_concerns':'Unknown', 'foundation_color':0, 'review':'', 'recommend':0,
                'unhelpful':0, 'helpful':0}
    details=info.split('\n')
    seperations['username']=details[0]
    seperations['helpful']=details[-1][-4:-1].strip('(').strip(' (')
    seperations['unhelpful']=details[-2][-4:-1].strip(' (')
    if details[-3]=='Recommends this product':
        seperations['recommend']=1
    else:
        seperations['recommend']=0
    seperations['review']=max(details, key=len)
    for value in details:
        if value[0:3]=='Eye':
            seperations['eye_color']=value[10:]
        elif value[0:4]=='Hair':
            seperations['hair']=value[11:]
        elif value[0:9]=='Skin Tone':
            seperations['skin_tone']=value[10:]
        elif value[0:9]=='Skin Type':
            seperations['skin_type']=value[10:]
        elif value[0:13]=='Skin concerns':
            seperations['skin_concerns']=value[14:]
        elif value[0:5]=='Color':
            seperations['foundation_color']=value[7:]
    df=df.append(seperations, ignore_index=True)

time: 12 s


In [18]:
df['star']=temp_df['star'].map(lambda x: x.strip('%;').strip('width: '))

time: 10.6 ms


In [19]:
df

,username,eye_color,hair,skin_tone,skin_type,skin_concerns,foundation_color,review,recommend,unhelpful,helpful,star
0,shaaliyah,Brown,Black,Deep,Combination,Acne,370,Since purchasing this foundation it's been my ...,1,0,42,100
1,bloodyblondie,Blue,Black,Porcelain,Dry,Blackheads,100,I was so nervous about trying this product bec...,1,0,10,100
2,Shortcake92,Brown,Black,Deep,Oily,Blackheads,410,Wonderful coverage with just one pump. Went on...,1,1,46,100
3,JDhome89,Brown,Brunette,Light,Oily,Acne,190,"If you’re oily, this foundation is amazing! Fu...",1,1,23,100
4,creecreex,Brown,Brunette,Deep,Normal,Cellulite,370,I have 370 and I got matched at Sephora. I lik...,1,1,20,100
5,Meowwkikii,Blue,Blonde,Porcelain,Combination,Sensitivity,110,I have rosacea and very fair skin. This is the...,1,1,20,100
6,sarina05,Brown,Brunette,Medium,Combination,Acne,240,THIS FOUNDATION IS AMAZZING. I have lots of ac...,1,15,286,100
7,comfortrose,Brown,Black,Ebony,Oily,Acne,490,There aren’t many mainstream brands that cater...,1,3,57,100
8,cocoabean16,Brown,Black,Dark,Combination,Blackheads,430,This is an awesome product! I wear this each a...,1,5,95,100
9,outerspaceglow,Green,Brunette,Porcelain,Combination,Acne,120,I've had this foundation for a little while no...,1,4,69,80


time: 91.1 ms


In [ ]:
# with open('final.pkl', 'wb') as picklefile:
#     pickle.dump(df, picklefile)